In [4]:
from __future__ import print_function
import os
import re
import logging
import SimpleITK as sitk
import radiomics
from radiomics import featureextractor
import numpy as np
import pandas as pd
import time

In [5]:
oar_names = ['spleen', 'kidney_right', 'kidney_left', 'gallbladder', 
             'liver', 'stomach', 'pancreas', 'prostate' ]
oar_indices = [1, 2, 3, 4, 
              5, 6, 7, 8]
external_directory = '/Volumes/My Passport/RPT/Patientwise_data_original'
local_directory = 'Radiomics features trial2'

Trial code - get radiomics features for one subject
All doses if number of timepoints == 3

In [6]:
# Define your radiomic feature classes
radiomic_features = ['firstorder', 'shape', 'ngtdm']

# Define your settings for feature extraction
settings = {
    'binWidth': 40,
    'resampledPixelSpacing': None,
    'interpolator': sitk.sitkLinear,
    'geometryTolerance': 1e-1,
    'correctMask': False
}

extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.disableAllFeatures()
for feature_class in radiomic_features:
    extractor.enableFeatureClassByName(feature_class)

In [19]:
general_seg = sitk.GetArrayFromImage(sitk.ReadImage('general_seg.nii'))
oar_coords = np.where(general_seg == 8)
mask_array = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(external_directory, 
                                                'P00011/P00011_D1_24_CT_seg.nii')))
oar_mask = np.zeros_like(mask_array, dtype=np.int8)

In [22]:
oar_mask[oar_coords]=1

In [23]:
def oar_wise_features(subject_file_path, subject_seg_path, subject_id):
    feature_results = {}
    
    image = sitk.ReadImage(subject_file_path)
    mask = sitk.ReadImage(subject_seg_path)
    mask.SetOrigin(image.GetOrigin())
    mask_array = sitk.GetArrayFromImage(mask)
    print(f"--------------Running file {subject_file_path}--------------")
    
    for oar_index in oar_indices:
        oar_features = {}
        if oar_index not in np.unique(mask_array):
            print(f'{oar_index} is not in {subject_id}, using general segmentation')
            oar_mask = np.zeros_like(mask_array, dtype=np.int8)
            oar_coords = np.where(general_seg == oar_index)
            oar_mask[oar_coords]=1
            print('Created alternate OAR mask')
        else:
            oar_mask = np.where(mask_array == oar_index, 1, 0).astype(np.int8)
            
        oar_mask = sitk.GetImageFromArray(oar_mask)
        oar_mask.CopyInformation(image)
        
        try:
            start_time = time.time()
            features = extractor.execute(image, oar_mask)
            time_taken = time.time() - start_time
            oar_features = {}
            count_features = 0
            for feature_name, feature_value in features.items():
                if 'diagnostics' not in feature_name:
                    count_features += 1
                    feature_type = feature_name.split('_')[0]
                    feature_label = feature_name.split('_')[-1]
                    oar_features[f"{feature_type}_{feature_label}"] = feature_value
            feature_results[oar_index] = oar_features
            print(f"Ran OAR {oar_index} in {time_taken} seconds")
        except Exception as e:
            feature_results[oar_index] = np.zeros(37)
            print(f"Error processing OAR {oar_index}: exception {e}")
                
        
    return feature_results

In [24]:
### For SPECT only

In [25]:
# spect_file_pattern = re.compile(rf"{subject_id}_D(\d+)_(\d+)_SPECT.nii")
# Iterate through each subject folder
for subject_folder in os.listdir(external_directory):
    if subject_folder == 'P00002':
        continue
    if subject_folder == 'P00007':
        continue
    
    subject_path = os.path.join(external_directory, subject_folder)
    subject_id = subject_folder
    if not os.path.isdir(subject_path):
        continue
    print(f"Currently processing {subject_path}")
        

    for file in os.listdir(subject_path):
        if file.endswith("_SPECT_reg.nii"):
            file_path = os.path.join(subject_path, file)
            seg = file.replace("_SPECT_reg.nii", "_CT_seg.nii")
            seg_path = os.path.join(subject_path, seg)
            if os.path.exists(seg_path):
                print(f"Found corresponding CT segmentation for SPECT file {file}")
                try:
                    print(f"Trying feature extraction for {file}")
                    start = time.time()
                    subject_radiomic_features = oar_wise_features(file_path, seg_path, 
                                                                  subject_id)
                    
                    subject_radiomic_features = pd.DataFrame.from_dict(subject_radiomic_features,
                                                                      orient='index').transpose()
                    save_path = os.path.join(local_directory, subject_folder)
                    if not os.path.exists(save_path):
                        os.makedirs(save_path)
                    
                    features_filename = file.replace("_SPECT_reg.nii", "_radiomic_features.pkl")
                    subject_radiomic_features.to_pickle(os.path.join(save_path, 
                                                                     features_filename))
                    time_taken = time.time() - start
                    print(f"Ran and saved {features_filename} in {time_taken} seconds")
                except Exception as e:
                    print(f"Error in {file}: exception {e}")

Currently processing /Volumes/My Passport/RPT/Patientwise_data_original/P00009
Currently processing /Volumes/My Passport/RPT/Patientwise_data_original/P00011
Found corresponding CT segmentation for SPECT file P00011_D1_120_SPECT_reg.nii
Trying feature extraction for P00011_D1_120_SPECT_reg.nii
--------------Running file /Volumes/My Passport/RPT/Patientwise_data_original/P00011/P00011_D1_120_SPECT_reg.nii--------------
Ran OAR 1 in 2.580443859100342 seconds
Ran OAR 2 in 2.1278648376464844 seconds
Ran OAR 3 in 2.031092882156372 seconds
Ran OAR 4 in 1.6258130073547363 seconds
Ran OAR 5 in 11.201428174972534 seconds
Ran OAR 6 in 1.8908159732818604 seconds
Ran OAR 7 in 1.7515699863433838 seconds
8 is not in P00011, using general segmentation
Created alternate OAR mask
Ran OAR 8 in 1.6080200672149658 seconds
Ran and saved P00011_D1_120_radiomic_features.pkl in 28.46593403816223 seconds
Found corresponding CT segmentation for SPECT file P00011_D1_24_SPECT_reg.nii
Trying feature extraction for

Created alternate OAR mask
Ran OAR 8 in 1.5931658744812012 seconds
Ran and saved P00012_D5_48_radiomic_features.pkl in 24.435811758041382 seconds
Found corresponding CT segmentation for SPECT file P00012_D6_24_SPECT_reg.nii
Trying feature extraction for P00012_D6_24_SPECT_reg.nii
--------------Running file /Volumes/My Passport/RPT/Patientwise_data_original/P00012/P00012_D6_24_SPECT_reg.nii--------------
Ran OAR 1 in 1.9507510662078857 seconds
Ran OAR 2 in 1.9111359119415283 seconds
Ran OAR 3 in 1.9551939964294434 seconds
Ran OAR 4 in 1.593677043914795 seconds
Ran OAR 5 in 7.328453063964844 seconds
Ran OAR 6 in 2.222965955734253 seconds
Ran OAR 7 in 1.8648929595947266 seconds
8 is not in P00012, using general segmentation
Created alternate OAR mask
Ran OAR 8 in 1.6251027584075928 seconds
Ran and saved P00012_D6_24_radiomic_features.pkl in 24.661396026611328 seconds
Found corresponding CT segmentation for SPECT file P00012_D6_48_SPECT_reg.nii
Trying feature extraction for P00012_D6_48_SP

Ran OAR 4 in 3.3987069129943848 seconds
Ran OAR 5 in 11.607694864273071 seconds
Ran OAR 6 in 4.160401821136475 seconds
Ran OAR 7 in 3.454432964324951 seconds
Ran OAR 8 in 3.3957881927490234 seconds
Ran and saved P00014_D5_24_radiomic_features.pkl in 44.65143084526062 seconds
Found corresponding CT segmentation for SPECT file P00014_D6_168_SPECT_reg.nii
Trying feature extraction for P00014_D6_168_SPECT_reg.nii
--------------Running file /Volumes/My Passport/RPT/Patientwise_data_original/P00014/P00014_D6_168_SPECT_reg.nii--------------
Ran OAR 1 in 3.6599037647247314 seconds
Ran OAR 2 in 3.6528079509735107 seconds
Ran OAR 3 in 3.632366895675659 seconds
Ran OAR 4 in 3.3744068145751953 seconds
Ran OAR 5 in 9.418320894241333 seconds
Ran OAR 6 in 5.240047216415405 seconds
Ran OAR 7 in 3.494391918182373 seconds
Ran OAR 8 in 3.409759998321533 seconds
Ran and saved P00014_D6_168_radiomic_features.pkl in 42.97142791748047 seconds
Found corresponding CT segmentation for SPECT file P00014_D6_48_SP

--------------Running file /Volumes/My Passport/RPT/Patientwise_data_original/P00016/P00016_D3_120_SPECT_reg.nii--------------
Ran OAR 1 in 3.753298044204712 seconds
Ran OAR 2 in 3.5362839698791504 seconds
Ran OAR 3 in 3.570697069168091 seconds
Ran OAR 4 in 3.3517568111419678 seconds
Ran OAR 5 in 6.6313557624816895 seconds
Ran OAR 6 in 3.4059557914733887 seconds
Ran OAR 7 in 3.4020516872406006 seconds
Ran OAR 8 in 3.325096845626831 seconds
Ran and saved P00016_D3_120_radiomic_features.pkl in 37.66453981399536 seconds
Found corresponding CT segmentation for SPECT file P00016_D3_72_SPECT_reg.nii
Trying feature extraction for P00016_D3_72_SPECT_reg.nii
--------------Running file /Volumes/My Passport/RPT/Patientwise_data_original/P00016/P00016_D3_72_SPECT_reg.nii--------------
Ran OAR 1 in 3.654336929321289 seconds
Ran OAR 2 in 3.459434986114502 seconds
Ran OAR 3 in 3.556943893432617 seconds
Ran OAR 4 in 3.3310301303863525 seconds
Ran OAR 5 in 7.454164028167725 seconds
Ran OAR 6 in 3.40972

Ran OAR 3 in 3.5948219299316406 seconds
Ran OAR 4 in 3.3489108085632324 seconds
Ran OAR 5 in 7.227090835571289 seconds
Ran OAR 6 in 3.726655960083008 seconds
Ran OAR 7 in 3.4310319423675537 seconds
Ran OAR 8 in 3.3821141719818115 seconds
Ran and saved P00017_D2_24_radiomic_features.pkl in 39.180562019348145 seconds
Found corresponding CT segmentation for SPECT file P00017_D2_48_SPECT_reg.nii
Trying feature extraction for P00017_D2_48_SPECT_reg.nii
--------------Running file /Volumes/My Passport/RPT/Patientwise_data_original/P00017/P00017_D2_48_SPECT_reg.nii--------------
Ran OAR 1 in 3.7096261978149414 seconds
Ran OAR 2 in 3.5544939041137695 seconds
Ran OAR 3 in 3.5463991165161133 seconds
Ran OAR 4 in 3.3791327476501465 seconds
Ran OAR 5 in 7.930883169174194 seconds
Ran OAR 6 in 4.111521005630493 seconds
Ran OAR 7 in 3.448815107345581 seconds
Ran OAR 8 in 3.315033197402954 seconds
Ran and saved P00017_D2_48_radiomic_features.pkl in 40.29423999786377 seconds
Found corresponding CT segme

...

Ran OAR 1 in 4.328537940979004 seconds
Ran OAR 2 in 3.7401421070098877 seconds
Ran OAR 3 in 3.8879501819610596 seconds
Ran OAR 4 in 3.6056408882141113 seconds
Ran OAR 5 in 10.220512866973877 seconds
Ran OAR 6 in 4.7616801261901855 seconds
Ran OAR 7 in 3.558943033218384 seconds
Ran OAR 8 in 3.589143753051758 seconds
Ran and saved P00033_D1_96_radiomic_features.pkl in 44.398759841918945 seconds
Found corresponding CT segmentation for SPECT file P00033_D2_24_SPECT_reg.nii
Trying feature extraction for P00033_D2_24_SPECT_reg.nii
--------------Running file /Volumes/My Passport/RPT/Patientwise_data_original/P00033/P00033_D2_24_SPECT_reg.nii--------------
Ran OAR 1 in 3.9302780628204346 seconds
Ran OAR 2 in 3.6845791339874268 seconds
Ran OAR 3 in 3.8104538917541504 seconds
Ran OAR 4 in 3.4366939067840576 seconds
Ran OAR 5 in 10.584720849990845 seconds
Ran OAR 6 in 5.247140884399414 seconds
Ran OAR 7 in 3.650308847427368 seconds
Ran OAR 8 in 3.522691011428833 seconds
Ran and saved P00033_D2_24

Ran OAR 1 in 3.761244297027588 seconds
Ran OAR 2 in 3.563422918319702 seconds
Ran OAR 3 in 3.5634682178497314 seconds
Ran OAR 4 in 3.448079824447632 seconds
Ran OAR 5 in 11.565209865570068 seconds
Ran OAR 6 in 3.8560068607330322 seconds
Ran OAR 7 in 3.580261707305908 seconds
Ran OAR 8 in 3.6794309616088867 seconds
Ran and saved P00035_D2_48_radiomic_features.pkl in 44.184407234191895 seconds
Found corresponding CT segmentation for SPECT file P00035_D2_72_SPECT_reg.nii
Trying feature extraction for P00035_D2_72_SPECT_reg.nii
--------------Running file /Volumes/My Passport/RPT/Patientwise_data_original/P00035/P00035_D2_72_SPECT_reg.nii--------------
Ran OAR 1 in 3.650007963180542 seconds
Ran OAR 2 in 3.659605026245117 seconds
Ran OAR 3 in 3.752845048904419 seconds
Ran OAR 4 in 3.529681921005249 seconds
Ran OAR 5 in 10.831219911575317 seconds
Ran OAR 6 in 3.930177688598633 seconds
Ran OAR 7 in 3.624922752380371 seconds
Ran OAR 8 in 3.458500862121582 seconds
Ran and saved P00035_D2_72_radi

Ran OAR 4 in 3.3899588584899902 seconds
Ran OAR 5 in 8.24845004081726 seconds
Ran OAR 6 in 4.307633876800537 seconds
Ran OAR 7 in 4.1677820682525635 seconds
Ran OAR 8 in 3.392327070236206 seconds
Ran and saved P00039_D1_120_radiomic_features.pkl in 40.9218909740448 seconds
Found corresponding CT segmentation for SPECT file P00039_D1_144_SPECT_reg.nii
Trying feature extraction for P00039_D1_144_SPECT_reg.nii
--------------Running file /Volumes/My Passport/RPT/Patientwise_data_original/P00039/P00039_D1_144_SPECT_reg.nii--------------
Ran OAR 1 in 3.5956149101257324 seconds
Ran OAR 2 in 3.5574557781219482 seconds
Ran OAR 3 in 3.6416239738464355 seconds
Ran OAR 4 in 3.524240016937256 seconds
Ran OAR 5 in 9.326218843460083 seconds
Ran OAR 6 in 4.704457759857178 seconds
Ran OAR 7 in 3.9549989700317383 seconds
Ran OAR 8 in 3.558095693588257 seconds
Ran and saved P00039_D1_144_radiomic_features.pkl in 43.32881307601929 seconds
Found corresponding CT segmentation for SPECT file P00039_D2_120_SP

Ran OAR 3 in 3.820570945739746 seconds
Ran OAR 4 in 3.392141103744507 seconds
Ran OAR 5 in 13.151777029037476 seconds
Ran OAR 6 in 4.105525970458984 seconds
Ran OAR 7 in 3.583491086959839 seconds
Ran OAR 8 in 3.337451696395874 seconds
Ran and saved P00042_D1_144_radiomic_features.pkl in 45.90367007255554 seconds
Found corresponding CT segmentation for SPECT file P00042_D1_48_SPECT_reg.nii
Trying feature extraction for P00042_D1_48_SPECT_reg.nii
--------------Running file /Volumes/My Passport/RPT/Patientwise_data_original/P00042/P00042_D1_48_SPECT_reg.nii--------------
Ran OAR 1 in 3.6409993171691895 seconds
Ran OAR 2 in 3.747300148010254 seconds
Ran OAR 3 in 3.7508530616760254 seconds
Ran OAR 4 in 3.3663220405578613 seconds
Ran OAR 5 in 14.075659036636353 seconds
Ran OAR 6 in 3.826408863067627 seconds
Ran OAR 7 in 3.5366201400756836 seconds
Ran OAR 8 in 3.385632038116455 seconds
Ran and saved P00042_D1_48_radiomic_features.pkl in 46.54872703552246 seconds
Currently processing /Volumes/

--------------Running file /Volumes/My Passport/RPT/Patientwise_data_original/P00047/P00047_D1_24_SPECT_reg.nii--------------
Ran OAR 1 in 3.785335063934326 seconds
Ran OAR 2 in 3.803644895553589 seconds
Ran OAR 3 in 3.9401371479034424 seconds
Ran OAR 4 in 3.4048409461975098 seconds
Ran OAR 5 in 9.474143981933594 seconds
Ran OAR 6 in 3.9358913898468018 seconds
Ran OAR 7 in 3.6384618282318115 seconds
Ran OAR 8 in 3.4002718925476074 seconds
Ran and saved P00047_D1_24_radiomic_features.pkl in 42.5941219329834 seconds
Found corresponding CT segmentation for SPECT file P00047_D1_48_SPECT_reg.nii
Trying feature extraction for P00047_D1_48_SPECT_reg.nii
--------------Running file /Volumes/My Passport/RPT/Patientwise_data_original/P00047/P00047_D1_48_SPECT_reg.nii--------------
Ran OAR 1 in 3.692854166030884 seconds
Ran OAR 2 in 3.873541831970215 seconds
Ran OAR 3 in 3.9361841678619385 seconds
Ran OAR 4 in 3.452552080154419 seconds
Ran OAR 5 in 8.964660882949829 seconds
Ran OAR 6 in 4.10303783